# Chapter 10. Spark SQL

## How to Run Spark SQL Queries

### Spark's Programmatic SQL Interface

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
  .config("spark.sql.shuffle.partitions", "5")\
  .appName("Spark SQL")\
  .getOrCreate()

24/08/13 19:44:50 WARN Utils: Your hostname, Khanhs-MAC.local resolves to a loopback address: 127.0.0.1; using 192.168.0.12 instead (on interface en0)
24/08/13 19:44:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/13 19:44:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.sql("SELECT 1 + 1").show()

+-------+
|(1 + 1)|
+-------+
|      2|
+-------+



In [5]:
spark.read.json("../data/flight-data/json/2015-summary.json")\
  .createOrReplaceTempView("some_sql_view")

In [6]:
spark.sql("""
  SELECT DEST_COUNTRY_NAME, SUM(count)
  FROM some_sql_view GROUP BY DEST_COUNTRY_NAME
""").where("DEST_COUNTRY_NAME like 'S%'").where("`sum(count)` > 10")\
  .count()

12

In [7]:
spark.sql("""
  CREATE TABLE IF NOT EXISTS flights (
    DEST_COUNTRY_NAME STRING, 
    ORIGIN_COUNTRY_NAME STRING, 
    count LONG
  )
  USING JSON 
  OPTIONS (path '../data/flight-data/json/2015-summary.json')
""")

24/08/13 19:45:06 WARN HadoopFSUtils: The directory file:/Users/khanhnn/Developer/DE/spark/practice_spark/practice/data/flight-data/json/2015-summary.json was not found. Was it deleted very recently?


DataFrame[]

In [8]:
spark.sql("""
  CREATE TABLE IF NOT EXISTS flights_csv (
  DEST_COUNTRY_NAME STRING,
  ORIGIN_COUNTRY_NAME STRING COMMENT "remember, the US will be most prevalent",
  count LONG)
  USING csv OPTIONS (header true, path '../data/flight-data/csv/2015-summary.csv')
""")

24/08/13 19:45:06 WARN HadoopFSUtils: The directory file:/Users/khanhnn/Developer/DE/spark/practice_spark/practice/data/flight-data/csv/2015-summary.csv was not found. Was it deleted very recently?


DataFrame[]

In [9]:
spark.sql("""CREATE TABLE flights_from_select USING parquet AS SELECT * FROM flights""")

24/08/13 19:45:06 WARN HadoopFSUtils: The directory file:/Users/khanhnn/Developer/DE/spark/practice_spark/practice/data/flight-data/json/2015-summary.json was not found. Was it deleted very recently?


SparkRuntimeException: [LOCATION_ALREADY_EXISTS] Cannot name the managed table as `spark_catalog`.`default`.`flights_from_select`, as its associated location 'file:/Users/khanhnn/Developer/DE/spark/practice_spark/practice/spark-warehouse/flights_from_select' already exists. Please pick a different table name, or remove the existing location first.

In [ ]:
spark.sql("""CREATE TABLE partitioned_flights USING parquet PARTITIONED BY (DEST_COUNTRY_NAME)
  AS SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 5""")

DataFrame[]

## Create External Tables

In [ ]:
spark.sql("""DESCRIBE TABLE flights_csv""")

DataFrame[col_name: string, data_type: string, comment: string]

In [ ]:
spark.sql("""
CREATE VIEW IF NOT EXISTS nested_data AS
    SELECT (DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME) as country, count FROM flights""")

NameError: name 'spark' is not defined